In [37]:
from sklearn.metrics.pairwise import cosine_similarity
from resources.keybert._model import KeyBERT
from resources.keybert._mmr import mmr
from resources.keybert._maxsum import max_sum_similarity
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

# doc = """
#          Supervised learning is the machine learning task of learning a function that
#          maps an input to an output based on example input-output pairs. It infers a
#          function from labeled training data consisting of a set of training examples.
#          In supervised learning, each example is a pair consisting of an input object
#          (typically a vector) and a desired output value (also called the supervisory signal). 
#          A supervised learning algorithm analyzes the training data and produces an inferred function, 
#          which can be used for mapping new examples. An optimal scenario will allow for the 
#          algorithm to correctly determine the class labels for unseen instances. This requires 
#          the learning algorithm to generalize from the training data to unseen situations in a 
#          'reasonable' way (see inductive bias).
#       """
# kw_model = KeyBERT(model='paraphrase-mpnet-base-v2')
# keywords = kw_model.extract_keywords(doc, top_n = 20, use_mmr = False, use_maxsum = False)
# keywords

In [4]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer().fit([doc])
# tokenizer = cv.build_tokenizer()
# tokens = tokenizer(doc)
# indices = [index for index, token in enumerate(tokens) if token.lower() in [word.lower() for word, _ in keywords]]
doc = [doc]

In [3]:
from sentence_transformers import SentenceTransformer, models
from transformers import AutoTokenizer, AutoModel
word_embedding_model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-mpnet-base-v2')

In [11]:
import torch
sentence = ['Supervised learning is the machine following singer generalize specialize']

# tokens = word_embedding_model.encode(doc)
tokens_1 = word_embedding_model.encode(sentence)
print(tokens_1[0].shape)



tokens_2 = tokenizer(doc,  padding=True, truncation=True, return_tensors='pt')
print(tokens_2)
embed_2 = model(**tokens_2)
embed_2[0].shape
word_embeddings = embed_2[0][0]

# mean = torch.sum(embed_2[0][0], 0) / 7
# print(mean.shape)

print(word_embeddings.shape)

# print(tokens_1[0])
# print(mean)


(768,)
{'input_ids': tensor([[    0, 13592,  4087,  2007,  2000,  3702,  4087,  4712,  2001,  4087,
          1041,  3857,  2012,  7345,  2023,  7957,  2004,  2023,  6438,  2245,
          2010,  2746,  7957,  1015,  6438,  7693,  1016,  2013,  2003, 24400,
          1041,  3857,  2017, 12603,  2735,  2955,  5402,  2001,  1041,  2279,
          2001,  2735,  4977,  1016,  2003, 13592,  4087,  1014,  2173,  2746,
          2007,  1041,  3944,  5402,  2001,  2023,  7957,  4878,  1010,  4054,
          1041,  9211,  1011,  2002,  1041,  9063,  6438,  3647,  1010,  2040,
          2174,  2000, 26657,  4746,  1011,  1016,  1041, 13592,  4087,  9900,
         17912,  2019,  2000,  2735,  2955,  2002,  7141,  2023,  2003,  7516,
          5600,  3857,  1014,  2033,  2068,  2026,  2113,  2009, 12379,  2051,
          4977,  1016,  2023, 15506, 11971,  2101,  3503,  2009,  2000,  9900,
          2004, 11182,  5650,  2000,  2469, 10877,  2009, 16104, 12111,  1016,
          2027,  5946,  2000,  

In [18]:
word_embeddings.shape
word_embeddings.shape[0]

150

In [6]:
# GET LISTS OF TOKEN, WORD, EMBEDED
tokens = []
words = []    
embeds = []
for s in doc[0].split():
    tokens_2 = tokenizer(s,  padding=True, truncation=True, return_tensors='pt', add_special_tokens=True)
    tokens.append(tokens_2['input_ids'])
    words.append(s)
#     print(s, tokens_2['input_ids'][0].detach().numpy())

for t in tokens:
    embeds.append(model(t)[0][0])

In [7]:
use_cosine = True
use_mmr = False
use_maxsum = False


doc_embedding = word_embedding_model.encode(doc)
doc_embedding = doc_embedding.reshape(1,-1)

for i,emb in enumerate(embeds):
    if use_cosine:
        distances = cosine_similarity(doc_embedding, emb.detach().numpy())
    elif use_mmr:
        keywords = mmr(doc_embedding, embeds, words, len(words))       
    elif use_maxsum:
        keywords = max_sum_similarity(doc_embedding, candidate_embeddings, candidates, top_n, nr_candidates)
        
    print(i, distances, words[i])

0 [[0.6156653  0.40459758 0.5324532 ]] Supervised
1 [[0.46053317 0.28330284 0.33434132]] learning
2 [[0.18774089 0.14277029 0.18716344]] is
3 [[0.03363296 0.00811013 0.05059395]] the
4 [[0.22825322 0.163795   0.21776482]] machine
5 [[0.46053317 0.28330284 0.33434132]] learning
6 [[0.22263744 0.18759245 0.22125056]] task
7 [[0.13614835 0.0811941  0.14957328]] of
8 [[0.46053317 0.28330284 0.33434132]] learning
9 [[0.1404782  0.08980355 0.15823776]] a
10 [[0.3378607  0.24902734 0.3041065 ]] function
11 [[0.14413594 0.09433894 0.16364029]] that
12 [[0.11860865 0.12283566 0.12095562]] maps
13 [[0.1513292  0.08542646 0.15078826]] an
14 [[0.09598282 0.08897761 0.11556689]] input
15 [[0.10634488 0.06290447 0.07959208]] to
16 [[0.1513292  0.08542646 0.15078826]] an
17 [[0.10617644 0.09742798 0.13306907]] output
18 [[0.1670565  0.10061654 0.12335663]] based
19 [[0.02611665 0.01918069 0.03610475]] on
20 [[0.23099244 0.16088837 0.21965632]] example
21 [[0.14265653 0.1355115  0.17601678 0.1335805  

In [77]:
def compare_kbert(sentence):
    print(sentence)
    sentence = [sentence]
    
    # GET LISTS OF TOKEN, WORD, EMBEDED
    tokens = []
    token_mean = []
    words = []    
    embeds = []
    for s in sentence[0].split():
        tokens_2 = tokenizer(s,  padding=True, truncation=True, return_tensors='pt', add_special_tokens=True)
        tokens.append(tokens_2['input_ids'])
        words.append(s)
    
        token_mean.append(word_embedding_model.encode(s))

    for t in tokens:
        embeds.append(model(t)[0][0])
    
    doc_embedding = word_embedding_model.encode(sentence)
    doc_embedding = doc_embedding.reshape(1,-1)

    for i,emb in enumerate(embeds):
        distances = cosine_similarity(doc_embedding, emb.detach().numpy()) 
        distances_mean = cosine_similarity(doc_embedding, token_mean[i].reshape(1, -1)) 
        print(i, 'Tokens:',distances[0], 'mean:', distances_mean[0], words[i])
        
    kw_model = KeyBERT(model='paraphrase-mpnet-base-v2')
    keywords = kw_model.extract_keywords(sentence, top_n = 20, use_mmr = False, use_maxsum = False)
    print('Keybert:')
    for k in keywords:
        print(k)

In [78]:
import pandas as pd
path = 'datasets/augmented/contextual_20/'

agnew = pd.read_csv(path + 'agnewsdataraw-8000')
biomedical = pd.read_csv(path + 'biomedical')
S = pd.read_csv(path + 'S')
search_snippets = pd.read_csv(path + 'search_snippets')
stackoverflow = pd.read_csv(path + 'stackoverflow')
T = pd.read_csv(path + 'T')
TS = pd.read_csv(path + 'TS')
tweet = pd.read_csv(path + 'tweet_remap_label')

In [79]:
compare_kbert(biomedical.text0[0])

histochemical and autoradiographic studies on the effects of aging on the mucopolysacharides of the periosteum
0 Tokens [0.31869373 0.33462316 0.3451317  0.31744963 0.28762853] mean: [0.35553688] histochemical
1 Tokens [0.07852793 0.07398125 0.08028755] mean: [0.09243661] and
2 Tokens [0.28436434 0.2733928  0.2836879  0.30796298 0.25719476 0.24439682] mean: [0.31404567] autoradiographic
3 Tokens [0.08418649 0.10184943 0.12347223] mean: [0.12261262] studies
4 Tokens [0.01807131 0.06152682 0.01235311] mean: [0.03772957] on
5 Tokens [0.01680208 0.05151888 0.05021342] mean: [0.04604571] the
6 Tokens [0.10784546 0.13083607 0.1288265 ] mean: [0.14562161] effects
7 Tokens [0.09656063 0.10012233 0.07405967] mean: [0.10394114] of
8 Tokens [0.3269264  0.27729267 0.34376478] mean: [0.36457142] aging
9 Tokens [0.01807131 0.06152682 0.01235311] mean: [0.03772957] on
10 Tokens [0.01680208 0.05151888 0.05021342] mean: [0.04604571] the
11 Tokens [0.3759539  0.40680844 0.42537028 0.41726905 0.42271626 

1it [00:00, 533.36it/s]

Keybert:
[('studies', 0.1226), ('effects', 0.1456), ('autoradiographic', 0.314), ('histochemical', 0.3555), ('aging', 0.3646), ('mucopolysacharides', 0.4525), ('periosteum', 0.5086)]


In [84]:
tokens_2 = tokenizer(biomedical.text0[0],  padding=True, truncation=True, return_tensors='pt')
embed_2 = model(**tokens_2)
word_embeddings = embed_2[0][0]

doc_embedding = word_embedding_model.encode(biomedical.text0[0])
doc_embedding = doc_embedding.reshape(1,-1)
for i,word_emb in enumerate(word_embeddings):
    distances = cosine_similarity(doc_embedding, word_emb.reshape(1,-1).detach().numpy())
    print('token', i, distances)

token 0 [[0.8175616]]
token 1 [[0.7008468]]
token 2 [[0.64592934]]
token 3 [[0.69003457]]
token 4 [[0.85731816]]
token 5 [[0.70908093]]
token 6 [[0.77330244]]
token 7 [[0.7960419]]
token 8 [[0.78885853]]
token 9 [[0.7871845]]
token 10 [[0.8765137]]
token 11 [[0.8922559]]
token 12 [[0.7186593]]
token 13 [[0.8092655]]
token 14 [[0.77096415]]
token 15 [[0.83447695]]
token 16 [[0.87465346]]
token 17 [[0.75342846]]
token 18 [[0.7652398]]
token 19 [[0.74907553]]
token 20 [[0.7397249]]
token 21 [[0.7530749]]
token 22 [[0.7630266]]
token 23 [[0.7514391]]
token 24 [[0.8326885]]
token 25 [[0.8316096]]
token 26 [[0.7452169]]
token 27 [[0.72645235]]
token 28 [[0.7543379]]
token 29 [[0.7568393]]
token 30 [[0.8077047]]
